In [1]:
# Datset link
# https://archive.ics.uci.edu/dataset/53/iris
# https://scikit-learn.org/dev/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [29]:
#load DS
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
df.head()

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0

In [30]:
#Split Dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])
X_train, y_train = train_df.drop(columns=['target']), train_df['target']
X_test, y_test = test_df.drop(columns=['target']), test_df['target']

In [31]:
#Normalise
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [32]:
#Convert to Tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [33]:
#Create custom NN
class IrisClassifier(nn.Module):
  def __init__(self, input_dim, hidden_dim,output_dim):
    super(IrisClassifier, self).__init__()
    self.network = nn.Sequential(
        nn.Linear(input_dim,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,output_dim),
    )

  def forward(self, x):
    return self.network(x)


In [34]:
#set Model parameters
input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 3

model = IrisClassifier(input_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [35]:
# Train the model

epochs = 500
for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()

  predictions = model(X_train_tensor)
  loss = criterion(predictions, y_train_tensor)

  loss.backward()
  optimizer.step()

  if (epoch+1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{epochs}], loss : {loss.item():.4f}")

Epoch [10/500], loss : 0.7896
Epoch [20/500], loss : 0.4530
Epoch [30/500], loss : 0.3318
Epoch [40/500], loss : 0.2531
Epoch [50/500], loss : 0.1717
Epoch [60/500], loss : 0.0968
Epoch [70/500], loss : 0.0602
Epoch [80/500], loss : 0.0466
Epoch [90/500], loss : 0.0405
Epoch [100/500], loss : 0.0372
Epoch [110/500], loss : 0.0351
Epoch [120/500], loss : 0.0332
Epoch [130/500], loss : 0.0311
Epoch [140/500], loss : 0.0291
Epoch [150/500], loss : 0.0270
Epoch [160/500], loss : 0.0247
Epoch [170/500], loss : 0.0220
Epoch [180/500], loss : 0.0195
Epoch [190/500], loss : 0.0168
Epoch [200/500], loss : 0.0143
Epoch [210/500], loss : 0.0120
Epoch [220/500], loss : 0.0099
Epoch [230/500], loss : 0.0081
Epoch [240/500], loss : 0.0067
Epoch [250/500], loss : 0.0055
Epoch [260/500], loss : 0.0046
Epoch [270/500], loss : 0.0038
Epoch [280/500], loss : 0.0032
Epoch [290/500], loss : 0.0028
Epoch [300/500], loss : 0.0024
Epoch [310/500], loss : 0.0021
Epoch [320/500], loss : 0.0019
Epoch [330/500], 

In [36]:
# Model evaluation on test
model.eval()
with torch.no_grad():
  y_pred = model(X_test_tensor)
  y_pred_labels = torch.argmax(y_pred, dim=1)

  accuracy = (y_pred_labels == y_test_tensor).sum().item() / y_test_tensor.size(0)
  print(f"Test accuracy : {accuracy:.4f}")

Test accuracy : 0.9333


In [37]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [38]:
def predict_iris(sepal_length, sepal_width, petal_length, petal_width):
  input_data = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
  input_data = scaler.transform(input_data)
  input_tesnor = torch.tensor(input_data, dtype = torch.float32)

  model.eval()
  with torch.no_grad():
    prediction = model(input_tesnor)
    predicted_class = torch.argmax(prediction, dim=1).item()
  return iris.target_names[predicted_class]

In [39]:
predicted_class = predict_iris(5.1, 3.5, 1.4, 0.2)
print(f"Predicted species : {predicted_class}")

Predicted species : setosa


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
